### Import Libraries

In [2]:
import os

import matplotlib.pyplot as plt

import json
import cv2
import numpy as np

from torchvision import transforms

from utilities import AITEXPatchedSegmentation

In [3]:
# Define paths
root = os.path.abspath(os.path.join(os.getcwd(), ".."))
model_dir = os.path.join(root, "models")
data_dir = os.path.join(root, "data")
aitex_dir = os.path.join(data_dir, "aitex")
od_dir = os.path.join(aitex_dir, "object_detection")
img_dir = os.path.join(od_dir, "images")
mask_dir = os.path.join(od_dir, "masks")

# Load dataset with transforms and split
transform = transforms.Compose([])
data = AITEXPatchedSegmentation(aitex_dir, transform=transform)#, normal_only=True)

In [4]:
def combine_bounding_boxes(bounding_boxes):
    """Combine into single bounding box."""
    x_min = [bbox[0] for bbox in bounding_boxes]
    y_min = [bbox[1] for bbox in bounding_boxes]
    x_max = [bbox[2] for bbox in bounding_boxes]
    y_max = [bbox[3] for bbox in bounding_boxes]

    return [(min(x_min), min(y_min), max(x_max), max(y_max))]

def get_bounding_boxes_from_mask(mask):
    """Detects contours from a mask and returns bounding boxes in COCO format."""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bbox = (x, y, x + w, y + h)
        bounding_boxes.append(bbox)

    if len(bounding_boxes) > 1:
        bounding_boxes = combine_bounding_boxes(bounding_boxes)
    
    xmin, ymin, xmax, ymax = bounding_boxes[0]
    w = xmax - xmin
    h = ymax - ymin
    return bounding_boxes[0], w * h

### Prepare Data

In [5]:
defect_codes = {
    0: "Normal",
    2:	"Broken end",
    6:	"Broken yarn",
    10:	"Broken pick",
    16:	"Weft curling",
    19:	"Fuzzyball",
    22:	"Cut selvage",
    23:	"Crease",
    25:	"Warp ball",
    27:	"Knots",
    29:	"Contamination",
    30: "Nep",
    36:	"Weft crack",
}

index_to_code = {i: val for i, val in enumerate(defect_codes.keys())}
code_to_index = {val: i for i, val in index_to_code.items()}

In [6]:
annotations = {
    "info": {
        "description": "AITEX Fabric Defects",
        "version": "1.0",
        "year": 2023,
        "date_created": "2023-07-08"
    },
    "categories": [{"id": index, "name": name, "supercategory": "object"} for index, (key, name) in enumerate(defect_codes.items())],
    "images": [],
    "annotations": []
}

# Process each image in the directory
for index, sample in enumerate(data):
    img = sample[0].numpy().reshape((256, 256))
    mask = sample[1].numpy().reshape((256, 256)).astype(np.uint8)
    class_label = sample[2]
    bbox, area = get_bounding_boxes_from_mask(mask)

    image_entry = {
        "id": index,
        "file_name": f"{index}.png",
        "width": img.shape[1],
        "height": img.shape[0]
    }
    annotations["images"].append(image_entry)
    cv2.imwrite(os.path.join(img_dir, image_entry["file_name"]), img * 255.)
    cv2.imwrite(os.path.join(mask_dir, image_entry["file_name"]), mask * 255.)
    annotation_entry = {
        "id": index,
        "image_id": index,
        "category_id": code_to_index[class_label],
        "bbox": tuple(bbox),
        "area": area,
        "iscrowd": 0
    }
    annotations["annotations"].append(annotation_entry)

In [80]:
with open(os.path.join(od_dir, "annotations.json"), "w") as anno_file:
    json.dump(annotations, anno_file)

### Quick Sanity Test

In [81]:
sample = data[0]

img = sample[0].numpy().reshape((256, 256))
mask = sample[1].numpy().reshape((256, 256)).astype(np.uint8)

bounding_boxes, contours = get_bounding_boxes_from_mask(mask)

display_mask = mask.copy()
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(display_mask,(x,y),(x+w,y+h),(255, 255, 255),1)

plt.imshow(display_mask)